In [4]:
from datetime import datetime, timedelta
print(datetime.now())
print(timedelta(7))

2022-05-05 12:55:20.302121
7 days, 0:00:00


In [1]:
from datetime import datetime, timedelta
today1 = datetime.now() - timedelta(7)
tomorrow1 = datetime.now() - timedelta(6)
print(today1)
print(tomorrow1)

2022-04-28 12:53:03.477288
2022-04-29 12:53:03.477288


In [9]:
import pandas as pd
import numpy as np
import os.path
import os,sys
from datetime import datetime, timedelta
import pyodbc
import pymssql
import pypyodbc
import csv
import math
import logging
from dotenv import load_dotenv
load_dotenv()

logger = logging.getLogger("Automation")
logger.setLevel(logging.INFO)
if logger.hasHandlers():
    logger.handlers.clear()
file_handler = logging.FileHandler("Automation.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

logger.info("******** HeartBeat Automation Process *********")

'''if datetime.now().hour >= 23:
    today1 = datetime.now() - timedelta(12)
    tomorrow1 = datetime.now() - timedelta(11)
else:
    today1 = datetime.now()
    tomorrow1 = datetime.now() - timedelta(1)'''

today1 = datetime.now() - timedelta(7)
tomorrow1 = datetime.now() - timedelta(6)

today = datetime.strftime(today1, '%m/%d/%Y')
tomorrow = datetime.strftime(tomorrow1, '%m/%d/%Y')

today_log = datetime.strftime(today1, '%d/%m/%Y')
tomorrow_log = datetime.strftime(tomorrow1, '%d/%m/%Y')

print(today)
print(tomorrow)

logger.info("Pax prediction for date {} started.".format(today_log))

driver = os.getenv("DRIVER")
server = os.getenv("SERVER_PROD")
database = os.getenv("DATABASE")
uid = os.getenv("UID_PROD")
pwd = os.getenv("PWD_PROD")
print(driver)

#TB_ACTIVEFLIGHT
conn = pypyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)

active_flight_query = """SELECT * FROM TB_ACTIVEFLIGHT
                    where STAD >= '"""  + today + "' and STAD < '" + tomorrow + "'"

active_flight_data = pd.read_sql_query(active_flight_query, conn)

#TB_STAND
conn = pypyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)

Bay_standcode_query = """SELECT * FROM TB_STAND"""
Bay_standcode = pd.read_sql_query(Bay_standcode_query, conn)

active_flight_data['standcode'] = active_flight_data['standcode'].str.strip()
active_flight_data['BAY'] = active_flight_data['standcode']

active_flight = active_flight_data[(active_flight_data['modesequence'].isin([0,1])) & (~active_flight_data['remarkcodeflight'].isin(['FCL'])) & (active_flight_data['iataservicetypecode'].isin(['C','G','J']))]

selected_columns_1 = active_flight[['afskey','flightno','tailno','aircrafttypecode','leg','station1','station2','station3','station4','station5','station6','iataservicetypecode','stad','atad','standcode','BAY','categorycode','remarkcodeflight']]
today_active_flight= selected_columns_1.copy().reset_index(drop=True)

#Bay_StanCode Data
Bay_standcode = Bay_standcode.rename(columns={'bay':'BAY','terminal':'TERMINAL','oper_type':'Oper_Type'})
Bay_standcode['BAY'] = Bay_standcode['BAY'].str.strip()

'''passenger_flight_data = Bay_standcode[(Bay_standcode['Oper_Type'].isin(['Passenger','Cargo/Mail']))].reset_index(drop=True)
selected_columns_2 = passenger_flight_data[['BAY','Oper_Type','TERMINAL']]
passenger_flight = selected_columns_2.copy()'''

TB_Active_Flight = pd.merge(today_active_flight,Bay_standcode,how='left',on=['BAY'])
#TB_Active_Flight = active_passenger_flight[(active_passenger_flight['Oper_Type'].isin(['Passenger']))].reset_index(drop=True)
Cargo_Flight = TB_Active_Flight[(TB_Active_Flight['Oper_Type'].isin(['Cargo/Mail']))].reset_index(drop=True)

TB_Active_Flight['Station'] = TB_Active_Flight['station1']+TB_Active_Flight['station2']+TB_Active_Flight['station3']+TB_Active_Flight['station4']+TB_Active_Flight['station5']+TB_Active_Flight['station6']
TB_Active_Flight['Station']= TB_Active_Flight['Station'].str.strip()
TB_Active_Flight['First_Station'] = TB_Active_Flight['Station'].str.slice(stop=3)
TB_Active_Flight['Last_Station'] = TB_Active_Flight['Station'].str.slice(start=-3)

TB_Active_Flight['First_Station'] = TB_Active_Flight['First_Station'].str.strip()
TB_Active_Flight['Last_Station'] = TB_Active_Flight['Last_Station'].str.strip()
TB_Active_Flight['routing'] = TB_Active_Flight.apply(lambda row:row['Last_Station'] if row['First_Station']=='KUL' else row['First_Station'],axis=1)

#required columns 
req_cols = ['afskey','flightno','tailno','stad','leg','aircrafttypecode','routing','categorycode','TERMINAL','remarkcodeflight','iataservicetypecode']
test_df = TB_Active_Flight[req_cols]
test_df.columns = ['afskey','fl_flight_no','fl_tail_registration','fl_actual_time','fl_leg','fl_aircraft_type','fl_routing','categorycode','TERMINAL','remarkcodeflight','iataservicetypecode']

#preprocessing

def weekinmonth(dates):
    """Get week number in a month.
    
    Parameters: 
        dates (pd.Series): Series of dates.
    Returns: 
        pd.Series: Week number in a month.
    """
    firstday_in_month = dates - pd.to_timedelta(dates.dt.day - 1, unit='d')
    return (dates.dt.day + firstday_in_month.dt.weekday) // 7 + 1

test_df['year'] = test_df['fl_actual_time'].dt.year
test_df['month'] = test_df['fl_actual_time'].dt.month
test_df['week'] = test_df['fl_actual_time'].dt.week
test_df['day'] = test_df['fl_actual_time'].dt.day
test_df['dayofweek'] = test_df['fl_actual_time'].dt.dayofweek
test_df['hour'] = test_df['fl_actual_time'].dt.hour
test_df['WOM'] = weekinmonth(test_df['fl_actual_time'])

conditions = [
    (test_df['hour'] < 8),
    (test_df['hour'] >= 8) & (test_df['hour'] < 15),
    (test_df['hour'] >= 15) & (test_df['hour'] < 21),
    (test_df['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

test_df['shift_time'] = np.select(conditions, values)

agg_data = pd.read_csv('pax_agg_post_covid_IATA.csv')

test_df = pd.merge(test_df,agg_data,how='left',on=['fl_tail_registration','fl_routing','fl_leg','categorycode','TERMINAL'])

test_df['mean'] = test_df['mean'].fillna(0)
test_df['min'] = test_df['min'].fillna(0)
test_df['max'] = test_df['max'].fillna(0)
test_df['median'] = test_df['median'].fillna(0)
test_df['last'] = test_df['last'].fillna(0)

logger.info("Test Dataframe shape is {}".format(test_df.shape))

unique_combo = pd.read_csv('Unique_combo_post_covid_IATA.csv')
unique_combo = unique_combo.drop(['fl_afskey'],axis=1)

test_df = pd.merge(test_df,unique_combo,how='left',on=['fl_tail_registration','fl_leg','fl_routing'])
test_df['Available'] = test_df['Available'].fillna('No')
test_df_yes = test_df[test_df['Available']=='Yes']
test_df_no = test_df[test_df['Available']=='No']
test_df.to_csv('test_df_post_covid_IATA.csv',index=False)

logger.info("Test Dataframe shape for Yes is {} and No is {}".format(test_df_yes.shape,test_df_no.shape))

#load the model
from pycaret.regression import *

saved_final_rf = load_model('Pax_Prediction_Model_post_covid_IATA')

new_prediction = predict_model(saved_final_rf, data=test_df_yes)

for_new_combo = pd.read_csv('for_new_combo_post_covid_IATA.csv')
for_new_combo = for_new_combo.rename(columns={'fl_type_count':'Label'})
new_prediction_mean = pd.merge(test_df_no,for_new_combo, how='left',on=['fl_leg','fl_routing'])
new_prediction = pd.concat([new_prediction,new_prediction_mean]).reset_index(drop=True)
new_prediction['Label'] = new_prediction['Label'].fillna(0)

new_prediction = new_prediction.sort_values('fl_actual_time',ascending=True).reset_index(drop=True)
new_prediction['Label'] = new_prediction.apply(lambda val: 0 if val.Label < 0 else val.Label,axis=1)
new_prediction['Label'] = new_prediction['Label'].apply(np.ceil).astype(int)

predicted_Pax = new_prediction.Label.sum()
logger.info("Predicted pax for {} is {}".format(today_log,predicted_Pax))
print(predicted_Pax)

pax_group = new_prediction.groupby(['shift_time','TERMINAL','categorycode','fl_leg'])['Label'].sum().reset_index().rename(columns={'Label':'Pax'})
pax_group.columns = ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Total_Pax_Optimal']

heartbeat_df = pd.read_excel("Heartbeat Analytics Dashboard Framework.xlsx", sheet_name = 'New Data Structure')

HeartBeat_Pax_Data = pd.merge(heartbeat_df,pax_group, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_Pax_Data = HeartBeat_Pax_Data.fillna(0)

#Passenger_Flight
TB_Active_Flight['hour'] = TB_Active_Flight['stad'].dt.hour

conditions = [
    (TB_Active_Flight['hour'] < 8),
    (TB_Active_Flight['hour'] >= 8) & (TB_Active_Flight['hour'] < 15),
    (TB_Active_Flight['hour'] >= 15) & (TB_Active_Flight['hour'] < 21),
    (TB_Active_Flight['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

TB_Active_Flight['shift_time'] = np.select(conditions, values)

pax_flight = TB_Active_Flight.groupby(['shift_time','TERMINAL','categorycode','leg'])['afskey'].count().reset_index().rename(columns={'afskey':'Pax_Flight'})

pax_flight.columns = ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Pax_Flight']

active_passenger_flights = pax_flight.Pax_Flight.sum()
logger.info("Active Passenger flight for {} is {}".format(today_log,active_passenger_flights))

HeartBeat_Pax_flight = pd.merge(heartbeat_df,pax_flight, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_Pax_flight = HeartBeat_Pax_flight.fillna(0)

#Cargo_Flight
Cargo_Flight['hour'] = Cargo_Flight['stad'].dt.hour

conditions = [
    (Cargo_Flight['hour'] < 8),
    (Cargo_Flight['hour'] >= 8) & (Cargo_Flight['hour'] < 15),
    (Cargo_Flight['hour'] >= 15) & (Cargo_Flight['hour'] < 21),
    (Cargo_Flight['hour'] >= 21)
    ]

values = ['0000 to 0800', '0800 to 1500', '1500 to 2100', '2100 to 2400']

Cargo_Flight['shift_time'] = np.select(conditions, values)

cargo_flight = Cargo_Flight.groupby(['shift_time','TERMINAL','categorycode','leg'])['afskey'].count().reset_index().rename(columns={'afskey':'Cargo_Flight'})

cargo_flight.columns = ['Shift_Time','Terminal','Domestic_or_International','Leg_Type','Cargo_Flight']

active_cargo_flights = cargo_flight.Cargo_Flight.sum()
logger.info("Active Cargo flight for {} is {}".format(today_log,active_cargo_flights))

HeartBeat_Cargo_flight = pd.merge(heartbeat_df,cargo_flight, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_Cargo_flight = HeartBeat_Cargo_flight.fillna(0)

#Check-In Counters

def cic_pred():
    """
    Predicts max CIC required for domestic and international pax respectively for KLIA and KLIA2 terminals.
    Done for each 8 hour window in a day. 
    Uses data from forecast of pax for that day
    """
    #!!!!!!!! FIX IN PRODUCTION !!!!!!!!!!
    #IMPORTANT ATTRIBUTES - Change after more analysis
    #% of pax arriving at 3, 2 and 1 hrs early
    PERCENTAGE_DEMAND = [20,70,10]
    #% of domestic pax using the CIC
    DOMESTIC_ACTUAL_PAX = 85
    # Avg processing time(mins) of a single pax. Calculated from extensive analysis of historical data
    PROC_TIME = 3
    
    #input data for cic prediction
    pax_forecast = new_prediction
    
    #dtype change for time column 
    pax_forecast['fl_actual_time'] = pd.to_datetime(pax_forecast['fl_actual_time'], format='%Y%m%d %H:%M:%S')
    pax_forecast.sort_values(by='fl_actual_time')    
    #Filtering for only departuring flights
    pax_forecast = pax_forecast[pax_forecast['fl_leg'] == 'D']
    
    #% of passesngers that would actually use cic
    def percentage_passengers_dom(each):
        return (each*DOMESTIC_ACTUAL_PAX)/100
    #Demand array in 24 hrs
    timer1 = [0]*24
    timer2 = [0]*24
    timer3 = [0]*24
    timer4 = [0]*24
    #Iterate thgough domestic flights
    for index, row in pax_forecast.iterrows():
        #hour index based on minutes of flight departure
        if pax_forecast['fl_actual_time'][index].minute >= 30 and pax_forecast['fl_actual_time'][index].hour != 23 :
            hour_index = pax_forecast['fl_actual_time'][index].hour +1 
        else:
            hour_index = pax_forecast['fl_actual_time'][index].hour
        hours_before_dep = 3
        if hour_index < 3:
            hours_before_dep = hour_index
        #Add demand to array based on % of demand of pax in a span of 3 hours
        for i in range(hours_before_dep):
            if row['TERMINAL'] == 'KLIA':
                if row['categorycode'] == 'D':
                    timer1[hour_index-i-1] += percentage_passengers_dom(pax_forecast['Label'][index])*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer2[hour_index-i-1] += pax_forecast['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
            else:
                if row['categorycode'] == 'D':
                    timer3[hour_index-i-1] += percentage_passengers_dom(pax_forecast['Label'][index])*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer4[hour_index-i-1] += pax_forecast['Label'][index]*(PERCENTAGE_DEMAND[i]/100)

    
    #Array with predicted no of CIC for each shift 
    #SHIFTS FOR A DAY
    #0000 - 0800
    #0800 - 1500
    #1500 - 2100
    #2100 - 2400
    col = [
    '0000 to 0800',
    '0800 to 1500',
    '1500 to 2100',
    '2100 to 2400',
    ]
    CIC = pd.DataFrame(columns = ['shifts','klia_domestic','klia_international','klia2_domestic','klia2_international'])
    
    CIC.loc[0] = [col[0],math.ceil((max(timer1[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer2[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer3[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer4[0:8])/6)*(PROC_TIME/10))]
    CIC.loc[1] = [col[1],math.ceil((max(timer1[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer2[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer3[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer4[8:15])/6)*(PROC_TIME/10))]
    CIC.loc[2] = [col[2],math.ceil((max(timer1[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer2[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer3[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer4[15:21])/6)*(PROC_TIME/10))]
    CIC.loc[3] = [col[3],math.ceil((max(timer1[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer2[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer3[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer4[21:24])/6)*(PROC_TIME/10))]
    return CIC

CIC = cic_pred()
CIC.columns = ['Shift_Time','KLIA_D','KLIA_I','klia2_D','klia2_I']
cic_summary = CIC.melt(id_vars='Shift_Time', var_name='Type',value_name='CIC_Optimal')
cic_summary[['Terminal','Domestic_or_International']] = cic_summary.Type.str.split('_',expand=True)
cic_summary = cic_summary.drop('Type',axis=1)
cic_summary['Leg_Type'] = 'D'

HeartBeat_CIC_Data = pd.merge(heartbeat_df,cic_summary, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_CIC_Data = HeartBeat_CIC_Data.fillna(0)

#Immigration Counters

def immigration_pred():
    """
    Predicts max immigration counters required for the 4 shifts in a day. 
    Uses data from forecast of pax for that day
    """
    #!!!!!!!! FIX IN PRODUCTION !!!!!!!!!!
    #IMPORTANT ATTRIBUTES - Change after more analysis
    #% of pax arriving at 3, 2 and 1 hrs early
    PERCENTAGE_DEMAND = [25,70,5]
    # Avg processing time(mins) of a single pax. Calculated from extensive analysis of historical data
    PROC_TIME = 1.5
    
    #input data for pax prediction
    pax_forecast = new_prediction
    
    #dtype change for time column 
    pax_forecast['fl_actual_time'] = pd.to_datetime(pax_forecast['fl_actual_time'], format='%Y%m%d %H:%M:%S')
    pax_forecast.sort_values(by='fl_actual_time')    
    #Filtering for only departuring flights
    #pax_forecast = pax_forecast[pax_forecast['fl_leg'] == 'D']
    #Filter international 
    pax_forecast_int = pax_forecast[pax_forecast['categorycode'] == 'I'] 
    pax_forecast_int = pax_forecast_int.reset_index()
    
    #Demand array in 24 hrs
    timer1 = [0]*24
    timer2 = [0]*24
    timer3 = [0]*24
    timer4 = [0]*24
    #Same as above but for international flights
    for index, row in pax_forecast_int.iterrows():
        if pax_forecast_int['fl_actual_time'][index].minute >= 30 and pax_forecast_int['fl_actual_time'][index].hour != 23 :
            hour_index = pax_forecast_int['fl_actual_time'][index].hour +1 
        else:
            hour_index = pax_forecast_int['fl_actual_time'][index].hour
        hours_before_dep = 3
        if hour_index < 3:
            hours_before_dep = hour_index
        for i in range(hours_before_dep):
            if row['TERMINAL'] == 'KLIA':
                if row['fl_leg'] == 'A':
                    timer1[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer2[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
            else:
                if row['fl_leg'] == 'A':
                    timer3[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
                else:
                    timer4[hour_index-i-1] += pax_forecast_int['Label'][index]*(PERCENTAGE_DEMAND[i]/100)
    #SHIFTS FOR A DAY
    #0000 - 0800
    #0800 - 1500
    #1500 - 2100
    #2100 - 2400
    col = [
    '0000 to 0800',
    '0800 to 1500',
    '1500 to 2100',
    '2100 to 2400',
    ]
    IMMIGRATION = pd.DataFrame(columns = ['shifts','klia_arrival','klia_departure','klia2_arrival','klia2_departure'])
    
    IMMIGRATION.loc[0] = [col[0],math.ceil((max(timer1[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer2[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer3[0:8])/6)*(PROC_TIME/10)),math.ceil((max(timer4[0:8])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[1] = [col[1],math.ceil((max(timer1[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer2[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer3[8:15])/6)*(PROC_TIME/10)),math.ceil((max(timer4[8:15])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[2] = [col[2],math.ceil((max(timer1[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer2[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer3[15:21])/6)*(PROC_TIME/10)),math.ceil((max(timer4[15:21])/6)*(PROC_TIME/10))]
    IMMIGRATION.loc[3] = [col[3],math.ceil((max(timer1[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer2[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer3[21:24])/6)*(PROC_TIME/10)),math.ceil((max(timer4[21:24])/6)*(PROC_TIME/10))]
    return IMMIGRATION

IMMIGRATION = immigration_pred()
IMMIGRATION.columns = ['Shift_Time','KLIA_A','KLIA_D','klia2_A','klia2_D']
immi_summary = IMMIGRATION.melt(id_vars='Shift_Time', var_name='Type',value_name='Immigration_Optimal')
immi_summary[['Terminal','Leg_Type']] = immi_summary.Type.str.split('_',expand=True)
immi_summary = immi_summary.drop('Type',axis=1)
immi_summary['Domestic_or_International'] = 'I'

HeartBeat_Immi_Data = pd.merge(heartbeat_df,immi_summary, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HeartBeat_Immi_Data = HeartBeat_Immi_Data.fillna(0)
HeartBeat_Immi_Data

#Combine All Dataframes

HB_Flight_Data = pd.merge(HeartBeat_Pax_flight,HeartBeat_Cargo_flight, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HB_Pax = pd.merge(HB_Flight_Data,HeartBeat_Pax_Data, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HB_Pax_CIC = pd.merge(HB_Pax,HeartBeat_CIC_Data, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])
HB_DF = pd.merge(HB_Pax_CIC,HeartBeat_Immi_Data, how='left',on=['Shift_Time','Terminal','Leg_Type','Domestic_or_International'])

today_date = datetime.strftime(today1, '%Y-%m-%d')
HB_DF['Date'] = today_date
first_column = HB_DF.pop('Date')
HB_DF.insert(0, 'Date', first_column)

#Rounding Values
HB_DF['Total_Pax_Optimal'] = HB_DF['Total_Pax_Optimal'].apply(np.ceil)
HB_DF['Pax_Flight'] = HB_DF['Pax_Flight'].astype('int64')
HB_DF['Cargo_Flight'] = HB_DF['Cargo_Flight'].astype('int64')
HB_DF['Total_Pax_Optimal'] = HB_DF['Total_Pax_Optimal'].astype('int64')

#Dummy Values for Dataframe
HB_DF['Security_Counter_Optimal'] = 200
HB_DF['BHS_Optimal'] = 150
HB_DF['KAT_Optimal'] = HB_DF.apply(lambda row:1 if row['Terminal']=='KLIA' else 0,axis=1)
HB_DF['PBB_Optimal'] = 60
HB_DF['PM_Optimal'] = 200
HB_DF['Pax_Washroom_Optimal'] = 90

#Dataframe for detailed prediction values
selected_columns = new_prediction[['afskey','fl_flight_no','fl_actual_time','fl_tail_registration','fl_aircraft_type','fl_routing','fl_leg','categorycode','TERMINAL','remarkcodeflight','Label']]
prediction_details = selected_columns.copy().reset_index(drop=True)
prediction_details.columns = ['afskey','fl_flight_no','fl_actual_time','fl_tail_registration','fl_aircraft_type','fl_routing','fl_leg','categorycode','TERMINAL','remarkcodeflight','predicted_pax']
prediction_details['afskey'] = prediction_details['afskey'].astype(int)
prediction_details['predicted_pax'] = prediction_details['predicted_pax'].apply(np.ceil).astype(int)

04/28/2022
04/29/2022
None


TypeError: can only concatenate str (not "NoneType") to str

In [3]:
print(prediction_details.shape)

(452, 11)


In [4]:
prediction_details['TERMINAL'] = prediction_details['TERMINAL'].replace(np.nan, 'None')

In [5]:
#Update consolidated data to MSSQL

import pyodbc
import pymssql
import pypyodbc

driver = os.getenv("DRIVER")
server = os.getenv("SERVER_UAT")
database = os.getenv("DATABASE")
uid = os.getenv("UID_UAT")
pwd = os.getenv("PWD_UAT")

conn = pyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)
cursor = conn.cursor()

'''cursor.execute("""delete from GCEO_HEARTBEAT where Date >='"""+today+"'")
conn.commit()

for index, row in HB_DF.iterrows():
    print(row)
    cursor.execute("INSERT INTO PSDB_OPS.dbo.GCEO_HEARTBEAT (Date, Shift_Time, Terminal, Leg_Type, Domestic_or_International, Pax_Flight, Cargo_Flight, Total_Pax_Optimal, CIC_Optimal, Immigration_Optimal, Security_Counter_Optimal, BHS_Optimal, KAT_Optimal, PBB_Optimal, PM_Optimal, Pax_Washroom_Optimal) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row.Date, row.Shift_Time, row.Terminal, row.Leg_Type, row.Domestic_or_International, row.Pax_Flight, row.Cargo_Flight, row.Total_Pax_Optimal, row.CIC_Optimal, row.Immigration_Optimal, row.Security_Counter_Optimal, row.BHS_Optimal, row.KAT_Optimal, row.PBB_Optimal, row.PM_Optimal, row.Pax_Washroom_Optimal)
conn.commit()'''

cursor.execute("""delete from GCEO_HBD_PC_IATA where fl_actual_time >='"""+today+"'")
conn.commit()

for index, row in prediction_details.iterrows():
    print(row)
    cursor.execute("INSERT INTO PSDB_OPS.dbo.GCEO_HBD_PC_IATA (afskey,fl_actual_time,fl_tail_registration,fl_aircraft_type,fl_routing,fl_leg,categorycode,TERMINAL,remarkcodeflight,predicted_pax) VALUES (?,?,?,?,?,?,?,?,?,?)",row.afskey,row.fl_actual_time,row.fl_tail_registration,row.fl_aircraft_type,row.fl_routing,row.fl_leg,row.categorycode,row.TERMINAL,row.remarkcodeflight,row.predicted_pax)
conn.commit()

logger.info("GCEO Heartbeat Automation was Successful for {}".format(today_log))



afskey                             10882815
fl_flight_no                        AK 5221
fl_actual_time          2022-04-28 00:00:00
fl_tail_registration                  9MAQH
fl_aircraft_type                        320
fl_routing                              KCH
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CLB
predicted_pax                           127
Name: 0, dtype: object
afskey                             10882982
fl_flight_no                        MH 2639
fl_actual_time          2022-04-28 00:15:00
fl_tail_registration                  9MMXQ
fl_aircraft_type                        73H
fl_routing                              BKI
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                           115
Name: 1, 

afskey                             10882317
fl_flight_no                        OD 0161
fl_actual_time          2022-04-28 05:00:00
fl_tail_registration                  9MLCR
fl_aircraft_type                        73H
fl_routing                              DAC
fl_leg                                    A
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            99
Name: 18, dtype: object
afskey                             10882870
fl_flight_no                        D7 0171
fl_actual_time          2022-04-28 05:10:00
fl_tail_registration                  9MXBE
fl_aircraft_type                        333
fl_routing                              BKK
fl_leg                                    A
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        COP
predicted_pax                           121
Name: 19

Name: 36, dtype: object
afskey                             10882091
fl_flight_no                        AK 0520
fl_actual_time          2022-04-28 06:55:00
fl_tail_registration                  9MAHE
fl_aircraft_type                        320
fl_routing                              SGN
fl_leg                                    D
categorycode                              I
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax                           104
Name: 37, dtype: object
afskey                             10881981
fl_flight_no                        AK 0380
fl_actual_time          2022-04-28 06:55:00
fl_tail_registration                  9MAGS
fl_aircraft_type                        32N
fl_routing                              CGK
fl_leg                                    D
categorycode                              I
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax               

afskey                             10882755
fl_flight_no                        MH 2547
fl_actual_time          2022-04-28 07:30:00
fl_tail_registration                  9MMXM
fl_aircraft_type                        73H
fl_routing                              KCH
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            90
Name: 54, dtype: object
afskey                             10881951
fl_flight_no                        D7 0376
fl_actual_time          2022-04-28 07:30:00
fl_tail_registration                  9MXXQ
fl_aircraft_type                        333
fl_routing                              TPE
fl_leg                                    D
categorycode                              I
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax                            38
Name: 55

afskey                             10883867
fl_flight_no                        MH 1384
fl_actual_time          2022-04-28 08:25:00
fl_tail_registration                  9MMXQ
fl_aircraft_type                        73H
fl_routing                              KBR
fl_leg                                    D
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           115
Name: 72, dtype: object
afskey                             10883514
fl_flight_no                        MH 2608
fl_actual_time          2022-04-28 08:25:00
fl_tail_registration                  9MMXW
fl_aircraft_type                        73H
fl_routing                              LBU
fl_leg                                    D
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           155
Name: 73

...

afskey                             10883973
fl_flight_no                        MH 0607
fl_actual_time          2022-04-28 20:05:00
fl_tail_registration                  9MMLR
fl_aircraft_type                        73H
fl_routing                              SIN
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           154
Name: 371, dtype: object
afskey                             10884589
fl_flight_no                        AK 6131
fl_actual_time          2022-04-28 20:05:00
fl_tail_registration                  9MRAV
fl_aircraft_type                        320
fl_routing                              PEN
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CLB
predicted_pax                           132
Name: 3

afskey                             10884605
fl_flight_no                        AK 6329
fl_actual_time          2022-04-28 20:45:00
fl_tail_registration                  9MRAK
fl_aircraft_type                        32N
fl_routing                              LGK
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CLB
predicted_pax                           120
Name: 391, dtype: object
afskey                             10884270
fl_flight_no                        MH 1281
fl_actual_time          2022-04-28 20:50:00
fl_tail_registration                  9MMXU
fl_aircraft_type                        73H
fl_routing                              KUA
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            73
Name: 3

afskey                             10884554
fl_flight_no                        AK 5197
fl_actual_time          2022-04-28 21:35:00
fl_tail_registration                  9MAQZ
fl_aircraft_type                        32A
fl_routing                              SDK
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CLB
predicted_pax                           120
Name: 410, dtype: object
afskey                             10884612
fl_flight_no                        AK 6451
fl_actual_time          2022-04-28 21:40:00
fl_tail_registration                  9MRCH
fl_aircraft_type                        320
fl_routing                              KBR
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CFB
predicted_pax                           126
Name: 4

afskey                             10884163
fl_flight_no                        MH 2597
fl_actual_time          2022-04-28 22:30:00
fl_tail_registration                  9MMXX
fl_aircraft_type                        73H
fl_routing                              KCH
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                           122
Name: 429, dtype: object
afskey                             10884388
fl_flight_no                        MH 2747
fl_actual_time          2022-04-28 22:30:00
fl_tail_registration                  9MMLT
fl_aircraft_type                        73H
fl_routing                              BTU
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                           119
Name: 4

afskey                             10883686
fl_flight_no                        MH 0123
fl_actual_time          2022-04-28 23:45:00
fl_tail_registration                  9MMTN
fl_aircraft_type                        333
fl_routing                              SYD
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           232
Name: 447, dtype: object
afskey                             10884854
fl_flight_no                       AK 5741A
fl_actual_time          2022-04-28 23:50:00
fl_tail_registration                  9MRAM
fl_aircraft_type                        32N
fl_routing                              TWU
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CCL
predicted_pax                           122
Name: 4

In [6]:
#Update consolidated data to MSSQL

import pyodbc
import pymssql
import pypyodbc

driver = os.getenv("DRIVER")
server = os.getenv("SERVER_PROD")
database = os.getenv("DATABASE")
uid = os.getenv("UID_PROD")
pwd = os.getenv("PWD_PROD")

conn = pyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)
cursor = conn.cursor()

#cursor.execute("""delete from GCEO_HEARTBEAT where Date >='"""+today+"'")
#conn.commit()

for index, row in HB_DF.iterrows():
    print(row)
    cursor.execute("INSERT INTO PSDB_OPS.dbo.GCEO_HEARTBEAT (Date, Shift_Time, Terminal, Leg_Type, Domestic_or_International, Pax_Flight, Cargo_Flight, Total_Pax_Optimal, CIC_Optimal, Immigration_Optimal, Security_Counter_Optimal, BHS_Optimal, KAT_Optimal, PBB_Optimal, PM_Optimal, Pax_Washroom_Optimal) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row.Date, row.Shift_Time, row.Terminal, row.Leg_Type, row.Domestic_or_International, row.Pax_Flight, row.Cargo_Flight, row.Total_Pax_Optimal, row.CIC_Optimal, row.Immigration_Optimal, row.Security_Counter_Optimal, row.BHS_Optimal, row.KAT_Optimal, row.PBB_Optimal, row.PM_Optimal, row.Pax_Washroom_Optimal)
conn.commit()

#cursor.execute("""delete from GCEO_HEARTBEAT_DETAILS where fl_actual_time >='"""+today+"'")
#conn.commit()

for index, row in prediction_details.iterrows():
    print(row)
    cursor.execute("INSERT INTO PSDB_OPS.dbo.GCEO_HEARTBEAT_DETAILS (afskey,fl_actual_time,fl_tail_registration,fl_aircraft_type,fl_routing,fl_leg,categorycode,TERMINAL,remarkcodeflight,predicted_pax) VALUES (?,?,?,?,?,?,?,?,?,?)",row.afskey,row.fl_actual_time,row.fl_tail_registration,row.fl_aircraft_type,row.fl_routing,row.fl_leg,row.categorycode,row.TERMINAL,row.remarkcodeflight,row.predicted_pax)
conn.commit()

logger.info("GCEO Heartbeat Automation was Successful for {}".format(today_log))

Date                           2021-11-30
Shift_Time                   0000 to 0800
Terminal                             KLIA
Leg_Type                                D
Domestic_or_International               D
Pax_Flight                              3
Cargo_Flight                            4
Total_Pax_Optimal                     377
CIC_Optimal                            25
Immigration_Optimal                     0
Security_Counter_Optimal              200
BHS_Optimal                           150
KAT_Optimal                             1
PBB_Optimal                            60
PM_Optimal                            200
Pax_Washroom_Optimal                   90
Name: 0, dtype: object
Date                           2021-11-30
Shift_Time                   0000 to 0800
Terminal                             KLIA
Leg_Type                                D
Domestic_or_International               I
Pax_Flight                              4
Cargo_Flight                            3
Total_Pax_O

Date                           2021-11-30
Shift_Time                   1500 to 2100
Terminal                            klia2
Leg_Type                                D
Domestic_or_International               D
Pax_Flight                             19
Cargo_Flight                            0
Total_Pax_Optimal                    2551
CIC_Optimal                            19
Immigration_Optimal                     0
Security_Counter_Optimal              200
BHS_Optimal                           150
KAT_Optimal                             0
PBB_Optimal                            60
PM_Optimal                            200
Pax_Washroom_Optimal                   90
Name: 12, dtype: object
Date                           2021-11-30
Shift_Time                   1500 to 2100
Terminal                            klia2
Leg_Type                                D
Domestic_or_International               I
Pax_Flight                              3
Cargo_Flight                            0
Total_Pax_

Name: 23, dtype: object
Date                           2021-11-30
Shift_Time                   0000 to 0800
Terminal                            klia2
Leg_Type                                A
Domestic_or_International               D
Pax_Flight                              1
Cargo_Flight                            0
Total_Pax_Optimal                     126
CIC_Optimal                             0
Immigration_Optimal                     0
Security_Counter_Optimal              200
BHS_Optimal                           150
KAT_Optimal                             0
PBB_Optimal                            60
PM_Optimal                            200
Pax_Washroom_Optimal                   90
Name: 24, dtype: object
Date                           2021-11-30
Shift_Time                   0000 to 0800
Terminal                            klia2
Leg_Type                                A
Domestic_or_International               I
Pax_Flight                              1
Cargo_Flight                

afskey                             10659333
fl_flight_no                        QR 0853
fl_actual_time          2021-11-30 02:45:00
fl_tail_registration                  A7BHE
fl_aircraft_type                        789
fl_routing                              DOH
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           207
Name: 6, dtype: object
afskey                             10659273
fl_flight_no                        MH 7980
fl_actual_time          2021-11-30 04:25:00
fl_tail_registration                  9MMTY
fl_aircraft_type                        332
fl_routing                              AMS
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                            52
Name: 7, 

afskey                             10660349
fl_flight_no                        AK 6130
fl_actual_time          2021-11-30 08:00:00
fl_tail_registration                  9MAQZ
fl_aircraft_type                        32A
fl_routing                              PEN
fl_leg                                    D
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax                           120
Name: 25, dtype: object
afskey                             10660674
fl_flight_no                        QR 0844
fl_actual_time          2021-11-30 08:05:00
fl_tail_registration                  A7BAA
fl_aircraft_type                        77W
fl_routing                              DOH
fl_leg                                    A
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            69
Name: 26

...

afskey                             10660487
fl_flight_no                        TK 0060
fl_actual_time          2021-11-30 17:25:00
fl_tail_registration                  TCJJT
fl_aircraft_type                        77W
fl_routing                              IST
fl_leg                                    A
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            71
Name: 183, dtype: object
afskey                             10660359
fl_flight_no                        AK 6446
fl_actual_time          2021-11-30 17:30:00
fl_tail_registration                  9MAFE
fl_aircraft_type                        320
fl_routing                              KBR
fl_leg                                    D
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax                           148
Name: 1

Name: 201, dtype: object
afskey                             10660421
fl_flight_no                        MH 1397
fl_actual_time          2021-11-30 18:55:00
fl_tail_registration                  9MMXG
fl_aircraft_type                        73H
fl_routing                              KBR
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            95
Name: 202, dtype: object
afskey                             10660346
fl_flight_no                        AK 6014
fl_actual_time          2021-11-30 18:55:00
fl_tail_registration                  9MAQZ
fl_aircraft_type                        32A
fl_routing                              AOR
fl_leg                                    D
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax             

afskey                             10660737
fl_flight_no                        MH 1455
fl_actual_time          2021-11-30 19:40:00
fl_tail_registration                  9MMXN
fl_aircraft_type                        73H
fl_routing                              LGK
fl_leg                                    A
categorycode                              D
TERMINAL                               KLIA
remarkcodeflight                        CLB
predicted_pax                            99
Name: 221, dtype: object
afskey                             10660185
fl_flight_no                        MH 0127
fl_actual_time          2021-11-30 19:40:00
fl_tail_registration                  9MMTM
fl_aircraft_type                        333
fl_routing                              PER
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                            25
Name: 2

afskey                             10660340
fl_flight_no                        AK 5140
fl_actual_time          2021-11-30 20:50:00
fl_tail_registration                  9MRAE
fl_aircraft_type                        32A
fl_routing                              BKI
fl_leg                                    D
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        FDP
predicted_pax                           124
Name: 239, dtype: object
afskey                             10659982
fl_flight_no                        EK 0448
fl_actual_time          2021-11-30 20:55:00
fl_tail_registration                  A6ECI
fl_aircraft_type                        77W
fl_routing                              AKL
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                            83
Name: 2

afskey                             10660151
fl_flight_no                        CZ 3048
fl_actual_time          2021-11-30 22:20:00
fl_tail_registration                  B20D7
fl_aircraft_type                        789
fl_routing                              CAN
fl_leg                                    D
categorycode                              I
TERMINAL                               KLIA
remarkcodeflight                        FDP
predicted_pax                           183
Name: 257, dtype: object
afskey                             10660781
fl_flight_no                        AK 6129
fl_actual_time          2021-11-30 22:30:00
fl_tail_registration                  9MRAG
fl_aircraft_type                        32A
fl_routing                              PEN
fl_leg                                    A
categorycode                              D
TERMINAL                              klia2
remarkcodeflight                        CLB
predicted_pax                           136
Name: 2

In [3]:
from datetime import datetime, timedelta
today1 = datetime.now() - timedelta(14)
tomorrow1 = datetime.now() - timedelta(13)
    
today = datetime.strftime(today1, '%m/%d/%Y')
tomorrow = datetime.strftime(tomorrow1, '%m/%d/%Y')

print(today)
print(tomorrow)

11/30/2021
12/01/2021


In [6]:
active_passenger_flight.head()

,afskey,flightno,tailno,aircrafttypecode,leg,station1,station2,station3,station4,station5,station6,stad,atad,standcode,BAY,categorycode,remarkcodeflight,Oper_Type,TERMINAL
0,10599262.0,QR 0853,A7BCA,788,D,KUL,DOH,,,,,2021-10-05 03:15:00,2021-10-05 03:25:00,C002,C002,I,FDP,Passenger,KLIA
1,10599947.0,BR 0228,B16732,77W,D,KUL,TPE,,,,,2021-10-05 15:30:00,2021-10-05 15:22:00,F002,F002,I,FDP,Cargo/Mail,KLIA
2,10599218.0,EK 0449,A6ECU,77W,D,KUL,DXB,,,,,2021-10-05 01:40:00,2021-10-05 01:57:00,C001,C001,I,FDP,Passenger,KLIA
3,10599253.0,IX 0623,VTAXJ,738,D,KUL,TRZ,,,,,2021-10-05 06:45:00,2021-10-05 06:52:00,C006,C006,I,FDP,Passenger,KLIA
4,10599542.0,EK 0449,A6ECU,77W,A,AKL,KUL,,,,,2021-10-05 00:05:00,2021-10-04 23:49:00,C001,C001,I,CLB,Passenger,KLIA


In [14]:
prediction_details.shape

(117, 11)

In [15]:
prediction_details.to_excel('pd.xlsx',index=False)

In [30]:
import pandas as pd
import numpy as np
import os.path
import os,sys
from datetime import datetime, timedelta
import pyodbc
import pymssql
import pypyodbc
import csv
import math
import logging
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
"""if datetime.now().hour >= 23:
    today1 = datetime.now() + timedelta(1)
    tomorrow1 = datetime.now() + timedelta(2)
else:
    today1 = datetime.now()
    tomorrow1 = datetime.now() + timedelta(1)"""

today1 = datetime.now() - timedelta(7)
tomorrow1 = datetime.now() - timedelta(6)
    
today = datetime.strftime(today1, '%m/%d/%Y')
tomorrow = datetime.strftime(tomorrow1, '%m/%d/%Y')

today_log = datetime.strftime(today1, '%d/%m/%Y')
tomorrow_log = datetime.strftime(tomorrow1, '%d/%m/%Y')

print(today)
print(tomorrow)

driver = os.getenv("DRIVER")
server = os.getenv("SERVER_PROD")
database = os.getenv("DATABASE")
uid = os.getenv("UID_PROD")
pwd = os.getenv("PWD_PROD")

conn = pypyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)

active_flight_query = """SELECT * FROM TB_ACTIVEFLIGHT where STAD >= '"""  + today + "' and STAD < '" + tomorrow + "'"

active_flight_data = pd.read_sql_query(active_flight_query, conn)


10/01/2021
10/02/2021


In [37]:
print(active_flight_data.shape)
active_flight_data.head()

(942, 86)


,afskey,flightno,leg,stad,sfskey,seasonid,officialflightdate,iataservicetypecode,flightmode,modesequence,...,messageid,deleteflag,memo,lastupdtime,lastupdsite,lastupduserid,lcctflightremark,mushowcheckinby,deskalloc,serviceclass
0,10601856.0,UA 7967,D,2021-10-08 07:10:00,None,W17,2021-10-08 07:10:00,H,3.0,2.0,...,None,None,,2021-10-06 00:02:49.197,FIDS,FIDAFS,None,None,None,
1,10601951.0,MH 9054,D,2021-10-08 03:15:00,None,W17,2021-10-08 03:15:00,J,3.0,2.0,...,None,None,,2021-10-06 00:02:49.530,FIDS,FIDAFS,None,None,None,
2,10602157.0,WY 5722,D,2021-10-08 06:50:00,None,W17,2021-10-08 06:50:00,J,3.0,4.0,...,None,None,,2021-10-08 00:17:12.940,FIDS,FIDDSPFLG,None,None,None,
3,10602248.0,NH 9894,D,2021-10-08 02:10:00,None,W17,2021-10-08 02:10:00,H,0.0,0.0,...,None,None,,2021-10-08 01:58:13.900,FIDS,FIDTMATMS,None,None,None,
4,10602403.0,MH 0195,A,2021-10-08 07:35:00,None,W17,2021-10-08 02:05:00,H,3.0,1.0,...,None,None,,2021-10-08 08:01:06.053,FIDS,FIDFCRCM2,None,None,None,


In [42]:
active_flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018 entries, 0 to 1017
Data columns (total 86 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   afskey                1018 non-null   float64       
 1   flightno              1018 non-null   object        
 2   leg                   1018 non-null   object        
 3   stad                  1018 non-null   datetime64[ns]
 4   sfskey                0 non-null      object        
 5   seasonid              1018 non-null   object        
 6   officialflightdate    1018 non-null   datetime64[ns]
 7   iataservicetypecode   1018 non-null   object        
 8   flightmode            1018 non-null   float64       
 9   modesequence          1018 non-null   float64       
 10  codeshareflight       1018 non-null   object        
 11  categorycode          1018 non-null   object        
 12  tailno                1018 non-null   object        
 13  aircrafttypecode  

In [43]:
today1 = datetime.now() - timedelta(7)
tomorrow1 = datetime.now() - timedelta(6)
    
today = datetime.strftime(today1, '%m/%d/%Y')
tomorrow = datetime.strftime(tomorrow1, '%m/%d/%Y')

today_log = datetime.strftime(today1, '%d/%m/%Y')
tomorrow_log = datetime.strftime(tomorrow1, '%d/%m/%Y')

print(today)
print(tomorrow)

driver = os.getenv("DRIVER")
server = os.getenv("SERVER_UAT")
database = os.getenv("DATABASE")
uid = os.getenv("UID_UAT")
pwd = os.getenv("PWD_UAT")

conn = pyodbc.connect("Driver={"+driver+"};Server="+server+";Database="+database+";uid="+uid+";pwd="+pwd)
cursor = conn.cursor()

active_flight_query_2 = """SELECT * FROM TB_ACTIVEFLIGHT where STAD >= '"""  + today + "' and STAD < '" + tomorrow + "'"

active_flight_data_2 = pd.read_sql_query(active_flight_query_2, conn)


10/01/2021
10/02/2021


In [44]:
active_flight_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1016 entries, 0 to 1015
Data columns (total 86 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   AFSKEY                1016 non-null   float64       
 1   FLIGHTNO              1016 non-null   object        
 2   LEG                   1016 non-null   object        
 3   STAD                  1016 non-null   datetime64[ns]
 4   SFSKEY                0 non-null      object        
 5   SEASONID              1016 non-null   object        
 6   OFFICIALFLIGHTDATE    1016 non-null   datetime64[ns]
 7   IATASERVICETYPECODE   1016 non-null   object        
 8   FLIGHTMODE            1016 non-null   float64       
 9   MODESEQUENCE          1016 non-null   float64       
 10  CODESHAREFLIGHT       1016 non-null   object        
 11  CATEGORYCODE          1016 non-null   object        
 12  TAILNO                1016 non-null   object        
 13  AIRCRAFTTYPECODE  

In [34]:
cursor.execute("""Delete FROM TB_ACTIVEFLIGHT where STAD >= '"""  + today + "' and STAD < '" + tomorrow + "'")
conn.commit()

In [35]:
for index, row in active_flight_data.iterrows():
    print(row)
    cursor.execute("INSERT INTO PSDB_OPS.dbo.TB_ACTIVEFLIGHT (afskey,flightno,leg,stad,sfskey,seasonid,officialflightdate,iataservicetypecode,flightmode,modesequence,codeshareflight,categorycode,tailno,aircrafttypecode,trafficright,aircraftversioncode,aircraftoperatorcode,pairafskey,emergencycode,mesgtype,irregularindicator,specialflightind,station1,std1,station2,sta2,std2,station3,sta3,std3,station4,sta4,std4,station5,sta5,std5,station6,sta6,endofflight,atmsetad,sitaetad,dispetad,atad,flightplanid,remarkcodeflight,flightdelaycode,checkintype,islandsalloc,schcounteropentime,schcounterclosetime,islandremark,standcode,standonchocktime,standoffchocktime,gateloungecode,prevgateloungecode,schgateopentime,schgateboardingtime,schgatefinalcalltime,schgateclosetime,remarkcodegate,useridatgate,actgateopentime,actgateboardingtime,actgatefinalcalltime,actgateclosetime,carouselcode,precarouselcode,remarkcodecarousel,useridatcarousel,carouseldelaycode,actcarouselopentime,actfirstbagtime,actlastbagtime,actcarouselclosetime,dispflag,messageid,deleteflag,memo,lastupdtime,lastupdsite,lastupduserid,lcctflightremark,mushowcheckinby,deskalloc,serviceclass) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",row.afskey, row.flightno, row.leg, row.stad, row.sfskey, row.seasonid, row.officialflightdate, row.iataservicetypecode, row.flightmode, row.modesequence, row.codeshareflight, row.categorycode, row.tailno, row.aircrafttypecode, row.trafficright, row.aircraftversioncode, row.aircraftoperatorcode, row.pairafskey, row.emergencycode, row.mesgtype, row.irregularindicator, row.specialflightind, row.station1, row.std1, row.station2, row.sta2, row.std2, row.station3, row.sta3, row.std3, row.station4, row.sta4, row.std4, row.station5, row.sta5, row.std5, row.station6, row.sta6, row.endofflight, row.atmsetad, row.sitaetad, row.dispetad, row.atad, row.flightplanid, row.remarkcodeflight, row.flightdelaycode, row.checkintype, row.islandsalloc, row.schcounteropentime, row.schcounterclosetime, row.islandremark, row.standcode, row.standonchocktime, row.standoffchocktime, row.gateloungecode, row.prevgateloungecode, row.schgateopentime, row.schgateboardingtime, row.schgatefinalcalltime, row.schgateclosetime, row.remarkcodegate, row.useridatgate, row.actgateopentime, row.actgateboardingtime, row.actgatefinalcalltime, row.actgateclosetime, row.carouselcode, row.precarouselcode, row.remarkcodecarousel, row.useridatcarousel, row.carouseldelaycode, row.actcarouselopentime, row.actfirstbagtime, row.actlastbagtime, row.actcarouselclosetime, row.dispflag, row.messageid, row.deleteflag, row.memo, row.lastupdtime, row.lastupdsite, row.lastupduserid, row.lcctflightremark, row.mushowcheckinby, row.deskalloc, row.serviceclass)
conn.commit()

afskey                      1.06031e+07
flightno                        AC 6257
leg                                   D
stad                2021-10-09 07:10:00
sfskey                             None
                           ...         
lastupduserid                    FIDAFS
lcctflightremark                   None
mushowcheckinby                    None
deskalloc                          None
serviceclass                           
Name: 0, Length: 86, dtype: object


DataError: ('22007', '[22007] [Microsoft][SQL Server Native Client 11.0][SQL Server]The conversion of a datetime2 data type to a datetime data type resulted in an out-of-range value. (242) (SQLExecDirectW); [22007] [Microsoft][SQL Server Native Client 11.0][SQL Server]The statement has been terminated. (3621)')

In [29]:

len(active_flight_data.columns)

86

In [25]:
active_flight_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 86 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   afskey                754 non-null    float64       
 1   flightno              754 non-null    object        
 2   leg                   754 non-null    object        
 3   stad                  754 non-null    datetime64[ns]
 4   sfskey                0 non-null      object        
 5   seasonid              754 non-null    object        
 6   officialflightdate    754 non-null    datetime64[ns]
 7   iataservicetypecode   754 non-null    object        
 8   flightmode            754 non-null    float64       
 9   modesequence          754 non-null    float64       
 10  codeshareflight       754 non-null    object        
 11  categorycode          754 non-null    object        
 12  tailno                754 non-null    object        
 13  aircrafttypecode    

In [22]:
print(active_flight_data_2.shape)
active_flight_data_2.head()

(754, 86)


,AFSKEY,FLIGHTNO,LEG,STAD,SFSKEY,SEASONID,OFFICIALFLIGHTDATE,IATASERVICETYPECODE,FLIGHTMODE,MODESEQUENCE,...,MESSAGEID,DELETEFLAG,MEMO,LASTUPDTIME,LASTUPDSITE,LASTUPDUSERID,LCCTFLIGHTREMARK,MUSHOWCHECKINBY,DESKALLOC,SERVICECLASS
0,10603081.0,NH 0816,D,2021-10-09 07:10:00,None,W17,2021-10-09 07:10:00,J,3.0,1.0,...,None,None,,2021-10-07 20:07:42.280,FIDS,FIDASM1,None,None,None,
1,10603108.0,J2 6411,D,2021-10-09 02:20:00,None,W17,2021-10-09 02:20:00,J,3.0,2.0,...,None,None,,2021-10-07 00:02:19.563,FIDS,FIDAFS,None,None,None,
2,10603173.0,IX 0623,D,2021-10-09 06:45:00,None,W17,2021-10-09 06:45:00,C,0.0,0.0,...,None,None,,2021-10-08 00:02:42.427,FIDS,FIDAFS,None,None,None,
3,10603183.0,QR 0853,D,2021-10-09 03:15:00,None,W17,2021-10-09 03:15:00,J,3.0,1.0,...,None,None,,2021-10-08 07:15:11.827,FIDS,FIDADMRAD,None,None,None,
4,10603229.0,FY 4172,D,2021-10-09 07:05:00,None,W17,2021-10-09 07:05:00,J,3.0,4.0,...,None,None,,2021-10-07 00:02:20.363,FIDS,FIDAFS,None,None,None,
